In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)  # Changed from Dropout2d to Dropout
        self.dropout2 = nn.Dropout(0.5)   # Changed from Dropout2d to Dropout
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


# Training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Testing function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%)\n')
    return accuracy

# Main function
def main():
    # Set up device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set up data loaders
    train_loader = DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True)

    test_loader = DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True)

    # Initialize the model
    model = MNISTNet().to(device)
    optimizer = optim.Adam(model.parameters())

    # Training loop
    for epoch in range(1, 9):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

    # Save the model
    torch.save(model.state_dict(), "mnist_cnn.pt")

    # Final test
    print("Loading the best model for final testing")
    model.load_state_dict(torch.load("mnist_cnn.pt"))
    final_accuracy = test(model, device, test_loader)
    print(f"Final Test Accuracy: {final_accuracy:.2f}%")

if __name__ == '__main__':
    main()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.326773
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.361670
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.134270
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.125308
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.087735
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.221354
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.036476
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.192336
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.078020
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.046846

Test set: Average loss: 0.0458, Accuracy: 9843/10000 (98.43%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.042218
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.058118
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.052294
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.050184
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.049351
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.053956
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.044703
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.173688
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.02572